In [ ]:
! unzip /content/drive/MyDrive/covid-dataset.zip -d /content/drive/MyDrive/coviddataset/

Streaming output truncated to the last 5000 lines.
  inflating: /content/data/Dataset/train/Normal/Normal-4118.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4119.png  
  inflating: /content/data/Dataset/train/Normal/Normal-412.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4120.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4121.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4122.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4123.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4124.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4125.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4126.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4127.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4128.png  
  inflating: /content/data/Dataset/train/Normal/Normal-4129.png  
  inflating: /content/data/Dataset/train/Normal/Normal-413.png  
  inflating: /content/data/

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Input

from tensorflow.keras.models import Model

from tensorflow.keras.preprocessing import image

from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input

from glob import glob

import numpy as np

import matplotlib.pyplot as plt

In [ ]:
imagesize=[299, 299]

trainPath=r"/content/data/COVID-19_Radiography_Dataset/train"




testPath= r"/content/data/COVID-19_Radiography_Dataset/test"



In [ ]:
train_datagen= ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip=True)

test_datagen=ImageDataGenerator (rescale = 1./255)


In [ ]:
training_set = train_datagen. flow_from_directory('/content/data/Dataset/train',
                                                    target_size=(299, 299),
                                                    batch_size = 16,
                                                    class_mode= 'categorical')

test_set = test_datagen.flow_from_directory('/content/data/Dataset/test',
                                                target_size=(299, 299),
                                                batch_size = 16,
                                                class_mode= 'categorical')

Found 19159 images belonging to 4 classes.
Found 2006 images belonging to 4 classes.


In [ ]:
from tensorflow.keras.applications import Xception
xception = Xception(input_shape=imagesize + [3], weights='imagenet', include_top=False)

In [ ]:
for layer in xception.layers:
  layer.trainable = False

In [ ]:
x = Flatten()(xception.output)

In [ ]:
prediction = Dense(4,activation='softmax')(x)

In [ ]:
model = Model(inputs=xception.input, outputs=prediction)

In [ ]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 block1_conv1 (Conv2D)          (None, 149, 149, 32  864         ['input_3[0][0]']                
                                )                                                                 
                                                                                                  
 block1_conv1_bn (BatchNormaliz  (None, 149, 149, 32  128        ['block1_conv1[0][0]']           
 ation)                         )                                                           

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
import tensorflow
my_callbacks=[
    tensorflow.keras.callbacks.EarlyStopping (patience=6)
]

In [ ]:
print(len(training_set))

1198


In [ ]:

r = model.fit_generator(
    training_set,
    validation_data=test_set,
    epochs=25,
    steps_per_epoch=len(training_set)//16,  # Adjusted value
    validation_steps=len(test_set)//16,  # Adjusted value
    callbacks=my_callbacks
)

<ipython-input-46-d143b3756e17>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/25
74/74 [==============================] - 36s 414ms/step - loss: 3.7261 - accuracy: 0.6672 - val_loss: 5.3828 - val_accuracy: 0.5714
Epoch 2/25
74/74 [==============================] - 30s 399ms/step - loss: 3.1680 - accuracy: 0.7306 - val_loss: 4.3970 - val_accuracy: 0.6964
Epoch 3/25
74/74 [==============================] - 30s 402ms/step - loss: 3.2648 - accuracy: 0.7416 - val_loss: 8.4068 - val_accuracy: 0.6607
Epoch 4/25
74/74 [==============================] - 30s 408ms/step - loss: 3.6055 - accuracy: 0.7601 - val_loss: 2.2885 - val_accuracy: 0.7679
Epoch 5/25
74/74 [==============================] - 30s 397ms/step - loss: 3.9218 - accuracy: 0.7475 - val_loss: 4.4717 - val_accuracy: 0.7232
Epoch 6/25
74/74 [==============================] - 30s 399ms/step - loss: 3.1289 - accuracy: 0.7753 - val_loss: 3.8198 - val_accuracy: 0.7054
Epoch 7/25
74/74 [==============================] - 30s 402ms/step - loss: 3.0753 - accuracy: 0.7804 - val_loss: 6.5845 - val_accuracy: 0.7232

In [ ]:
model.save("/content/drive/MyDrive/model/model.h5")

In [ ]:
import cv2

from tensorflow.keras.utils import load_img, img_to_array

from tensorflow.keras.applications. inception_v3 import decode_predictions

from tensorflow.keras.models import load_model
from tensorflow.keras.applications.xception import Xception

In [ ]:
model=load_model("/content/drive/MyDrive/model/model.h5")

In [ ]:
img = '/content/drive/MyDrive/images.jpg'

image = load_img(img, target_size=(299,299))

image=img_to_array(image)

image=image.reshape((1,299,299,3))

image.shape

(1, 299, 299, 3)

In [ ]:
pred=model.predict(image)

1/1 [==============================] - 2s 2s/step


In [ ]:
labels = ['COVID','Lung_opacity','Normal','Viral Pneumonia']

In [ ]:
import numpy as np

p=np.argmax(pred)


In [ ]:

labels[p]

'COVID'